In [20]:
import xgboost 
from xgboost import XGBClassifier

import numpy as np
from numpy import loadtxt

import librosa
import io
from ipywidgets import widgets
import warnings
warnings.filterwarnings('ignore')

In [19]:
model = XGBClassifier()
model.load_model("XBG_model.pth")

[21:36:42] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:854: Loading model from XGBoost < 1.0.0, consider saving it again for improved compatibility


<h1><center>Harry Potter or Star Wars

This web app based on machine learning is created to the classify a hymn/sound/tune either as "Harry Potter" or "Star Wars". To do so, click the upload button below, and select the the audio file. 

In [3]:
lbl_greet = widgets.Label('Select the Audio File')

btn_upload = widgets.FileUpload()

out_pl = widgets.Output()

lbl_pred = widgets.Label()

In [4]:
def on_data_change(change):
    lbl_pred.value = ''
    
    
    for name, file_info in btn_upload.value.items():
        audio, sample_rate = librosa.load(io.BytesIO(file_info['content']), res_type='kaiser_fast')
    #print('Uploaded audio')
    def mfcc_extractor(file_, sample_rate):
        mfccs_features = librosa.feature.mfcc(y=file_, sr=sample_rate, n_mfcc=64)
        mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
        return mfccs_scaled_features
        
    
    out_pl.clear_output()
    
    class_mapping = [
    "Star Wars",
    "Harry Potter"
    ]

    test_audio = mfcc_extractor(audio, sample_rate)
    test_audio.shape = -1,64

    # PREDICT 
    prediction_index = model.predict(np.array(test_audio.tolist()))
    prediction = class_mapping[prediction_index[0]]

    #print(f"Predicted: '{prediction}'")
    
    lbl_pred.value = f"Predicted: '{prediction}'"
    

In [5]:
btn_upload.observe(on_data_change, names=['data'])

In [6]:
display(lbl_greet, btn_upload, out_pl, lbl_pred)

Label(value='Select the Audio File')

FileUpload(value={}, description='Upload')

Output()

Label(value='')